### Data preprocessing

In [15]:
import pandas as pd
import numpy as np

import scipy.stats as ss
from sklearn.metrics import ndcg_score

In [16]:
train_df = pd.read_csv('/Users/nadys/recsys/hw01_data/train.csv')
test_df = pd.read_csv('/Users/nadys/recsys/hw01_data/test.csv')

In [17]:
np.random.seed(42)
train_df['rnd'] = np.random.random(len(train_df))

In [18]:
users = sorted(set(train_df['user']))
tracks = sorted(set(train_df['track']))
len(users), len(tracks)

(10000, 50000)

In [19]:
user_to_id = dict()
id_to_user = dict()
for i, u in enumerate(users):
    user_to_id[u] = i
    id_to_user[i] = u

In [20]:
train_users_ids = [user_to_id[u] for u in train_df['user']]
test_users_ids = [user_to_id[u] for u in test_df['user']]

In [21]:
train_df['user_id'] = train_users_ids
test_df['user_id'] = test_users_ids

## v11 SVD++

In [22]:
from collections import defaultdict
from surprise import SVDpp, Dataset, NormalPredictor, Reader, accuracy
from surprise.model_selection import train_test_split, cross_validate, KFold, GridSearchCV

In [23]:
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(train_df[['user_id', 'track', 'time']], reader)

In [24]:
# trainset, testset = train_test_split(data, test_size=0.1)

reg_all по дефолту 0.02 попробовать оставить done

v10: n_epochs=300, n_factors=80, reg_all=0.1

v11: n_epochs=400, n_factors=80

v61: n_epochs=500, n_factors=100

v62: n_epochs=600, n_factors=120

In [25]:
algo = SVDpp(n_epochs=800, n_factors=140)
algo.fit(data.build_full_trainset())

In [26]:
predictions = [
    algo.predict(uid, iid).est
    for _, (uid, iid) in test_df[['user_id', 'track']].iterrows()
]

In [27]:
np.count_nonzero(np.array(predictions)) / len(predictions)  # у какой части элементов матрицы получили что-то нормальное    

0.9373555247341655

# Exporting results

In [28]:
res_df = test_df[['user', 'track']].copy()
res_df['score'] = predictions
res_df.to_csv('submission_v63.csv', index=False)